# Crear dataset de entrada para predecir

In [29]:
import pandas as pd
import os
from unidecode import unidecode
import Levenshtein 
from difflib import get_close_matches
import re
import openpyxl

#### Leer esxcell mercado/plantilla

In [30]:
import pandas as pd

# Ruta al archivo Excel
archivo_excel = "../mi_plantilla2023-12-27--18-45-S.xlsx"

# Lee el archivo Excel con pandas y especifica que no hay encabezado
df = pd.read_excel(archivo_excel, header=None)

# Obtén los valores de la primera columna (columna 0)
valores_Mercado = df.iloc[:, 0].tolist()

# Imprime o utiliza los valores según sea necesario
print(valores_Mercado)
print(len(valores_Mercado))


['G. Mamardashvili', 'A. Pedraza', 'A. Rüdiger', 'E. García', 'Y. Couto', 'A. Baena', 'A. Tchouaméni', 'F. De Jong', 'M. Llorente', 'N. Maksimović', 'P. Torre', 'V. Tsygankov', 'Á. Morata', 'A. Douvikas', 'C. Ramos']
15


#### Función para encontrar los jugaodres de mi plantilla/mercado en el dataset de estadisticas

In [31]:
def encontrar_coincidencias(valores_Historico, valores_Mercado):
    print("Buscando coincidencias...\n")
    posiciones = []

    for pos, nombre_completo in enumerate(valores_Historico):
        coincidencias_iniciales = [nombre_abreviado for nombre_abreviado in valores_Mercado if nombre_abreviado.startswith(nombre_completo[0])]

        coincidencias_texto = get_close_matches(nombre_completo, coincidencias_iniciales)

        if coincidencias_texto:
            posiciones.append(pos)
            print(f"Coincidencia en {pos} para '{nombre_completo}': {coincidencias_texto[0]}")
    print("\n" + "." * 150+ "\n")
    return posiciones

#### Buscar jugaodres en los datasets de estadisticas que me interesan (jugaodres de mi plantilla / jugaodres en el mercado actual)

In [32]:
# Ruta a la carpeta que contiene los archivos Excel
carpeta_excel = "../data/test_precedecir"

# Lista global para almacenar todas las filas seleccionadas
filas_jugadores = []

# Lista todos los archivos en la carpeta con extensión .xlsx
archivos_excel = [archivo for archivo in os.listdir(carpeta_excel) if archivo.endswith(".xlsx")]

# Itera sobre cada archivo Excel en la carpeta
for archivo in archivos_excel:
    
    print("\n" + "=" * 150 + "\n")  # Separador visual entre archivos
    
    # Ruta al archivo Excel actual
    ruta_archivo = os.path.join(carpeta_excel, archivo)

    # Lee el archivo Excel con pandas y especifica que no hay encabezado
    df = pd.read_excel(ruta_archivo, header=None)

    # Obtén los valores de la primera columna (o cualquier columna que desees)
    valores_Historico = df.iloc[:, 0].tolist()

    # Llama a la función encontrar_coincidencias y actualiza los valores de los índices
    posiciones_coincidentes = encontrar_coincidencias(valores_Historico, valores_Mercado)
    
    # Accede a las filas correspondientes a los índices proporcionados
    filas_seleccionadas = df.iloc[posiciones_coincidentes]

    # Convierte las filas seleccionadas a una lista de diccionarios para un fácil acceso a los atributos
    datos_filas = filas_seleccionadas.to_dict(orient='records')

    # Agrega las filas seleccionadas a la lista global
    global filas_jugadores
    filas_jugadores.extend(datos_filas)

    # Imprime los datos de las filas seleccionadas
    for idx, fila in zip(posiciones_coincidentes, datos_filas):
        print(f"Coincidencia en {ruta_archivo}, fila {idx}: {fila}\n")
        
# Leer y almacenar cabecera
df = pd.read_excel(ruta_archivo, header=None, nrows=1)

# Almacena los nombres de las columnas en una lista
nombres_columnas = list(df.iloc[0])

# Imprime los nombres de las columnas
print("Nombres de las columnas:", nombres_columnas)



Buscando coincidencias...

Coincidencia en 120 para 'Anastasios Douvikas': A. Douvikas
Coincidencia en 125 para 'Adrià Pedrosa': A. Pedraza
Coincidencia en 148 para 'Nemanja Maksimović': N. Maksimović
Coincidencia en 162 para 'Chris Ramos': C. Ramos
Coincidencia en 210 para 'Álvaro Morata': Á. Morata
Coincidencia en 221 para 'Marcos Llorente': M. Llorente
Coincidencia en 247 para 'Eric García': E. García
Coincidencia en 249 para 'Yan Couto': Y. Couto
Coincidencia en 267 para 'Viktor Tsygankov': V. Tsygankov
Coincidencia en 274 para 'Pablo Torre': P. Torre
Coincidencia en 288 para 'Antonio Rüdiger': A. Rüdiger
Coincidencia en 324 para 'Aurélien Tchouaméni': A. Tchouaméni
Coincidencia en 374 para 'Frenkie De Jong': F. De Jong
Coincidencia en 380 para 'Giorgi Mamardashvili': G. Mamardashvili

......................................................................................................................................................

Coincidencia en ../data/test_precedecir\datas

#### Se crea un excell donde se almacenan todos los atributos procesados

In [54]:
#CREAR EXCELL FINAL PARA PASAR AL MODELO Y PREDECIR
nombre_archivo = "../data/test_precedecir/to_predict_dataset.xlsx"

# Verificar si el archivo existe y eliminarlo si es necesario
if os.path.exists(nombre_archivo):
    os.remove(nombre_archivo)

# Crear un nuevo libro de trabajo (Workbook)
wb = openpyxl.Workbook()

# Seleccionar la hoja activa
sheet = wb.active

# Agregar los nombres de las columnas a la primera fila de la hoja
sheet.append(nombres_columnas)

# Guardar el libro de trabajo en el archivo especificado
wb.save(nombre_archivo)

#### Acceder a las estadisticas de cada jugaodor del mercado/plantilla en todos los datasets(partidos de cada jornada de LaLiga) donde se han encontrado registros de este, para: 

- Generar medias, porcentajes... de valores numéricos estadisticos alamcenados en todos los dataset (Ej: media de goles)

- Acceder a la web de Mister Fantasy par inicializar ciertos atributos (Ej: proximo rival al que se enfrentará un jugaodor)

- Analizar atributos no numéricos alamcenados en todos los dataset para generar un nuevo atributo que de alguna forma represente todas las muestras en todos los datatsets (Ej: razón de ausencia de un jugaodor: jornada 1: Lesión, jornada 2: Suspension por acumulación de amarillas)

In [55]:
# Diccionario para almacenar los índices de las filas repetidas
indice_nombres = {}

# Iterar filas para las repetidas
for indice, fila_jugador in enumerate(filas_jugadores):
    # Obtiene el nombre del jugador (asumiendo que está en la primera posición)
    nombre_jugador = fila_jugador[0]
    
    # Verifica si el nombre ya ha sido visto
    if nombre_jugador in indice_nombres:
        indice_nombres[nombre_jugador].append(indice)
    else:
        indice_nombres[nombre_jugador] = [indice]

# Gestionar filas repetidas 
for nombre, indices in indice_nombres.items():
    print("\n" + "_" * 75+ "\n")
    print(f"Nombre: {nombre}, Índices: {indices}")

    # Acceder al nombre de cada índice repetido
    for indice in indices:
        nombre_repetido = filas_jugadores[indice][0]
        print(f"  Nombre repetido en el índice {indice}: {nombre_repetido}")
    
    print("\n" + "-" * 60+ "\n")
    # GENERAR DATAFRAME para cada jugador ###################################################################
    print("Datos del dataframe")
    
    ## NOMBRE #####
    nombre= filas_jugadores[indice][0]
    print("Nombre: ",)
    
    ## VALOR #####
    valor= 0
    for indice in indices:
        valor_jugador1 = str(filas_jugadores[indice][1])
        valor_duplicado = float(valor_jugador1.replace('.', ''))
        valor = valor + valor_duplicado
    valor= valor/len(indices)
    print("Valor:",valor)
    
    ## POSICIÓN #####
    pos_jugador=filas_jugadores[indice][2]
    print(f"Posición: {pos_jugador}")
        
    ## EQUIPO #####
    eq_jugador=filas_jugadores[indice][3]
    print(f"Equipo: {eq_jugador}")
    
    ## PUNTUACIÓN FANTASY, AS, MARCA, MD #####
    columnas = [4, 5, 6, 7]
    todos_puntos=[]
    
    for columna in columnas:
        puntos = 0

        for indice in indices:
            pnts = filas_jugadores[indice][columna]
            puntos += pnts
        
        puntos = puntos / len(indices)
        todos_puntos.append(puntos)
        nombre_columna = {4: "Fantasy", 5: "As", 6: "Marca", 7: "Mundo Deportivo"}[columna]

        print(f"Puntos {nombre_columna}: {puntos}")


## ULTIMO RIVAL #####
    ultimo_equipo_rival = 0
    print(f"Último equipo rival: {ultimo_equipo_rival}")

## RESULTADO DEL PARTIDO #####
    resultado_ultimo_partido = 0
    print(f"Resultado último partido: {resultado_ultimo_partido}")

## PROXIMO RIVAL #####
    proximo_equipo_rival = 0
    print(f"Próximo equipo rival: {proximo_equipo_rival}")

## PROXIMO PARTIDO ES LOCAL #####
    proximo_equipo_local = 0
    print(f"Próximo equipo como local: {proximo_equipo_local}")

## MEDIA EN CASA #####
    media_puntos_local = 0
    print(f"Media de puntos como local: {media_puntos_local}")

## MEDIA FUERA #####
    media_puntos_fuera = 0
    print(f"Media de puntos como visitante: {media_puntos_fuera}")

    
    
    # EDAD #####
    edad=filas_jugadores[indice][14]
    print(f"Edad: {edad}")
    
    # ALTURA #####
    try:
        altura_jugador = filas_jugadores[indice][15]
        altura_jugador = float(altura_jugador)
        print(f"Altura: {altura_jugador}")
    except (ValueError, TypeError):
        altura_jugador = filas_jugadores[indice][15].replace('m', '').replace(',', '.')
        altura_jugador = float(altura_jugador)
        print(f"Altura: {altura_jugador}")

    # PESO #####
    try:
        peso_jugador = filas_jugadores[indice][16]
        peso_jugador = float(peso_jugador)
        print(f"Peso: {peso_jugador}")
    except (ValueError, TypeError):
        peso_jugador = filas_jugadores[indice][16].replace('kg', '').replace(',', '.')
        peso_jugador = float(peso_jugador)
        print(f"Peso: {peso_jugador}")

    ## PUNTUACIÓN SF#####
    puntuacion= 0
    for indice in indices:
        pts=float(filas_jugadores[indice][17])
        puntuacion = puntuacion + pts   
    puntuacion= puntuacion/len(indices)
    print("Puntuación SF:",puntuacion)
    
    # MINUTES PLAYED #####
    minutes = 0
    for indice in indices:
        try:
            mn = int(filas_jugadores[indice][18])
        except (ValueError, TypeError):
            mn = int(str(filas_jugadores[indice][18]).replace("'", ''))

        minutes += mn

    minutes = minutes / len(indices)
    print("Minutos disputados:", minutes)

    
    ## EXPECTED ASSISTS 
    expassists = 0
    for indice in indices:
        expa = float(filas_jugadores[indice][19])
        expassists = expassists + expa
    expassists = expassists / len(indices)
    print("Provavilidad de que un pase sea asistencia:",expassists)
    
    ## ESTADÍSCAS PORTEROS #################################################################
    ## (XA), SAVES, GOALS PREVENTED, PUNCHES, RUNS OUT (SUCC.), HIGH CLAIMS #####
    saves = 0
    golasPrev = 0
    punches = 0
    runsOut=0
    hightClaims=0
    for indice in indices:        
        svs = float(filas_jugadores[indice][20])
        saves = saves+ svs
        
        gp = float(filas_jugadores[indice][21])
        golasPrev = golasPrev+ gp
        
        pch = float(filas_jugadores[indice][22])
        punches = punches+ pch
        
        rnsout_str = str(filas_jugadores[indice][23])
        # Eliminar paréntesis y el número dentro de ellos
        rnsout_str_cleaned = re.sub(r'\(\d+\)', '', rnsout_str).strip()
        rnsout = int(rnsout_str_cleaned) if rnsout_str_cleaned else 0

        
        hclaims = float(filas_jugadores[indice][24])
        hightClaims = hightClaims+ hclaims
        
    saves = saves / len(indices)
    print("Paradas (porteros):",saves)
    
    golasPrev = golasPrev / len(indices)
    print("Goles evitados (porteros):",golasPrev)
            
    punches = punches / len(indices)
    print("Despeje de puños (porteros):",punches)
    
    runsOut = runsOut / len(indices)
    print("Salida con balón esxitosa (porteros):",punches)
    
    hightClaims = hightClaims / len(indices)
    print("Captura de balon en altura exitosa (porteros):",hightClaims)
    
    ## TOUCHES #####
    touches=0
    for indice in indices:        
        touch = int(filas_jugadores[indice][25])
        touches = touches+ touch
        
    touches = touches / len(indices)
    print("Toques de balón:",touches)
        
    ## ACC. PASSES #####
    accpass = 0
    for indice in indices:        
        accp_match = re.search(r'\((\d+)%\)', str(filas_jugadores[indice][26]))
        accp = accp_match.group(1) if accp_match else None
        if accp:
            accpass += float(accp)

    accpass = accpass / len(indices) if len(indices) > 0 else 0
    print("Pases precisos:", accpass)
    
    ## KEY PASSES #####
    keypass=0
    for indice in indices:        
        keyp = int(filas_jugadores[indice][27])
        keypass = keypass+ keyp
        
    keypass = keypass / len(indices)
    print("Pases claves:",keypass)
    
    
## CROSSES (ACC.) #####
    crosses=0
    print(f"Centros precisos: {crosses}")

## LONG BALLS (ACC.) #####
    longball=0
    print(f"Pase largo preciso: {longball}")
    
## CLEARANCES #####
    clearances=0
    print(f"Ocasiones evitadas de gol: {clearances}")
    
## BLOCKED SHOTS #####
    blocketshoots=0
    print(f"Tiros bloqueados: {blocketshoots}")
    
## INTERCEPTIONS #####
    interceptions=0
    print(f"Pases nterceptados: {interceptions}")
    
## TACKLES #####
    tackles=0
    print(f"Robos de balón sin falta: {tackles}")
    
## DRIBBLED PAST #####
    dribbled=0
    print(f"Veces regateado: {dribbled}")
    
## GROUND DUELS (WON) #####
    groundduels=0
    print(f"Duelos en el suelo Ganados: {groundduels}")
    
## AERIAL DUELS (WON) #####
    airduels=0
    print(f"Duelos aereos ganados: {airduels}")
    
## FOULS #####
    fouls=0
    print(f"Faltas cometidas: {fouls}")
    
## WAS FOULED #####
    fouled=0
    print(f"Faltas recibidas: {fouled}")
    
## SHOTS ON TARGET #####
    tarhetshoot=0
    print(f"Tiros a puerta: {tarhetshoot}")
    
## SHOTS OFF TARGET #####
    offtargetshoot=0
    print(f"Tiros que no fueron a puerta: {offtargetshoot}")
    
## SHOTS BLOCKED #####
    blockedshoots=0
    print(f"Disparos realizados bloqueados : {blockedshoots}")
    
## DRIBBLE ATTEMPTS (SUCC.) #####
    dribbleatempts=0
    print(f"Regates exitosos: {dribbleatempts}")
    
## GOALS #####
    goals=0
    print(f"Goles: {goals}")
    
## ASSISTS #####
    assists=0
    print(f"Asistencias: {assists}")
    
## POSSESSION LOST #####
    posesion_perdida = 0
    print(f"Posesión perdida: {posesion_perdida}")
    
## EXPECTED GOALS (XG) #####
    xg = 0.0
    print(f"Provavilidad de que ls disparos se conviertan en goles: {xg}")
    
## PENALTY MISS #####
    penalty_miss = 0
    print(f"Penaltis fallados : {penalty_miss}")
    
## BIG CHANCES CREATED #####
    big_chances_created = 0
    print(f"Ocasiones de gol generadas: {big_chances_created}")
    
## PENALTY WON #####
    penalty_won = 0
    print(f"Penaltys conseguidos: {penalty_won}")
    
## BIG CHANCES MISSED #####
    big_chance_miss=0
    print(f"Ocasiones de gol desperdiciadas: {big_chance_miss}")
    
## SAVES FROM INSIDE BOX #####
    savesBox=0
    print(f"Ocasione sd e gol evitadas desde dentro del area: {savesBox}")
    
## PENALTY COMMITTED #####
    penaltyDone=0
    print(f"Penalties cometidos: {penaltyDone}")
    
## OFFSIDES #####
    oddside=0
    #print(f"Veces en fuera de juego: {oddside}")
    
## HIT WOODWORK #####
    palos=0
    print(f"Disparos al paloooo: {palos}")
    
## AUSENCIA #####
    ausencia=0
    print(f"Causa por no estar convocado: {ausencia}")
    
## ERROR LED TO SHOT #####
    shot_led_error=0
    print(f"Errores que derivan en un disparo del rival: {shot_led_error}")
    
## ERROR LED TO GOAL   ##### 
    goal_led_error=0
    print(f"Errores que derivan en un gol del rival: {goal_led_error}")
    
    #########################################################################################################
    print("\n")
    
    # Cargar excell
    wb = openpyxl.load_workbook(nombre_archivo)
    
    # Seleccionar la hoja activa
    sheet = wb.active
        
    # Lista de variables a almacenar
    nueva_fila = [
    nombre, valor, pos_jugador, eq_jugador,
    todos_puntos[0], todos_puntos[1], todos_puntos[2], todos_puntos[3], ultimo_equipo_rival, resultado_ultimo_partido, proximo_equipo_rival, proximo_equipo_local,
    media_puntos_local, media_puntos_fuera, edad, altura_jugador, peso_jugador, puntuacion, minutes, expassists, saves, golasPrev, punches, runsOut,
    hightClaims, touches, accpass, keypass, crosses, longball, tarhetshoot, offtargetshoot, blockedshoots, dribbleatempts, goals, assists, clearances,
    blocketshoots, interceptions, tackles, dribbled, groundduels, airduels, fouls, fouled, savesBox, penaltyDone, oddside, palos, ausencia, shot_led_error,
    goal_led_error, posesion_perdida, xg, penalty_miss, big_chances_created, penalty_won, big_chance_miss]

    
    # Escribir la nueva fila en la hoja de cálculo
    sheet.append(nueva_fila)

    # Guardar el archivo Excel
    wb.save(nombre_archivo)
    

print(f"Se han generado exitosamente el dataset en {ruta_excel_nuevo} para realizar la sprediciones sobre los jugadores selecionados.")


___________________________________________________________________________

Nombre: Anastasios Douvikas, Índices: [0, 14, 28, 42]
  Nombre repetido en el índice 0: Anastasios Douvikas
  Nombre repetido en el índice 14: Anastasios Douvikas
  Nombre repetido en el índice 28: Anastasios Douvikas
  Nombre repetido en el índice 42: Anastasios Douvikas

------------------------------------------------------------

Datos del dataframe
Nombre: 
Valor: 966000.0
Posición: DL
Equipo: Celta de Vigo
Puntos Fantasy: 2.0
Puntos As: 2.0
Puntos Marca: 2.0
Puntos Mundo Deportivo: 2.0
Último equipo rival: 0
Resultado último partido: 0
Próximo equipo rival: 0
Próximo equipo como local: 0
Media de puntos como local: 0
Media de puntos como visitante: 0
Edad: 24
Altura: 1.84
Peso: 78.0
Puntuación SF: 6.4
Minutos disputados: 9.0
Provavilidad de que un pase sea asistencia: 0.0
Paradas (porteros): 0.0
Goles evitados (porteros): 0.0
Despeje de puños (porteros): 0.0
Salida con balón esxitosa (porteros): 0.0
Cap